In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping

In [2]:
# Wrap Keras model in a function
def create_model(neurons_layer1=32, neurons_layer2=16, learning_rate=0.01):
    model = Sequential()
    model.add(Dense(neurons_layer1, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(neurons_layer2, activation='relu'))
    model.add(Dense(1, activation='relu'))  # Use 'relu' activation in the output layer
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [3]:
# Load the data
data = pd.read_csv('D:\\bachelor\\customer_36.csv')

# Preprocess the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

In [4]:
# Split the data into training and testing sets
X = data.drop('consumption', axis=1)
y = data['consumption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Split the data into training and testing sets
# Assuming the data is hourly, 24*7 rows would make a week
X_train, X_test = X[:-24*7], X[-24*7:]
y_train, y_test = y[:-24*7], y[-24*7:]

X = pd.DataFrame(scaler_X.fit_transform(X), columns=X.columns)
y = pd.DataFrame(scaler_y.fit_transform(y.values.reshape(-1,1)), columns=[y.name])

In [5]:
# Create a KerasRegressor with the model
model = KerasRegressor(build_fn=create_model, neurons_layer1=32, neurons_layer2=16, learning_rate=0.01, verbose=0)

In [6]:
param_grid = {
    'neurons_layer1': list(range(10, 310, 10)),  # 30 values from 10 to 300
    'neurons_layer2': list(range(10, 310, 10)),  # 30 values from 10 to 300
	'learning_rate': [0.1, 0.01, 0.001, 0.0001]  # example learning rates
}
# Create early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

d:\bachelor\BACH\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
d:\bachelor\BACH\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\bachelor\BACH\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.201984 using {'learning_rate': 0.001, 'neurons_layer1': 260, 'neurons_layer2': 10}
